In [1]:
from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#!pip install xlrd

In [3]:
xls_file = "default_credit_score.xls"
df = pd.read_excel(xls_file)

# Set the second row as header
df.columns = df.iloc[0]  # Assign the second row as header
df = df[1:].reset_index(drop=True)  # Remove the first row and reset index

# Save as CSV
csv_file = "default_credit_score.csv"
df.to_csv(csv_file, index=False)

print(f"File saved as {csv_file} with corrected headers.")

File saved as default_credit_score.csv with corrected headers.


In [4]:
df = pd.read_csv("default_credit_score.csv")
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [6]:
df.columns = [['ID', 'LIMIT_BAL', 'GENDER', 'EDUCATION', 'MARRIAGE', 'AGE', 
               'PAY_ST9', 'PAY_ST8', 'PAY_ST7', 'PAY_ST6', 'PAY_ST5','PAY_ST4',
               'BILL_AMT9', 'BILL_AMT8', 'BILL_AMT7', 'BILL_AMT6', 'BILL_AMT5', 'BILL_AMT4', 
               'PAY_AMT9', 'PAY_AMT8', 'PAY_AMT7', 'PAY_AMT6', 'PAY_AMT5', 'PAY_AMT4',
       'DEFAULT_PAYMENT']]

In [7]:
df.head()

,ID,LIMIT_BAL,GENDER,EDUCATION,MARRIAGE,AGE,PAY_ST9,PAY_ST8,PAY_ST7,PAY_ST6,...,BILL_AMT6,BILL_AMT5,BILL_AMT4,PAY_AMT9,PAY_AMT8,PAY_AMT7,PAY_AMT6,PAY_AMT5,PAY_AMT4,DEFAULT_PAYMENT
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [8]:
df.columns

MultiIndex([(             'ID',),
            (      'LIMIT_BAL',),
            (         'GENDER',),
            (      'EDUCATION',),
            (       'MARRIAGE',),
            (            'AGE',),
            (        'PAY_ST9',),
            (        'PAY_ST8',),
            (        'PAY_ST7',),
            (        'PAY_ST6',),
            (        'PAY_ST5',),
            (        'PAY_ST4',),
            (      'BILL_AMT9',),
            (      'BILL_AMT8',),
            (      'BILL_AMT7',),
            (      'BILL_AMT6',),
            (      'BILL_AMT5',),
            (      'BILL_AMT4',),
            (       'PAY_AMT9',),
            (       'PAY_AMT8',),
            (       'PAY_AMT7',),
            (       'PAY_AMT6',),
            (       'PAY_AMT5',),
            (       'PAY_AMT4',),
            ('DEFAULT_PAYMENT',)],
           )